In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("ProductRanking.com") \
      .getOrCreate()

23/11/06 23:36:53 WARN Utils: Your hostname, TruongHoang resolves to a loopback address: 127.0.1.1; using 192.168.81.115 instead (on interface wlp3s0)
23/11/06 23:36:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 23:36:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json("hdfs://localhost:9000/tiki/product_tiki.json")

In [3]:
df.printSchema()
df.count()

root
 |-- _corrupt_record: string (nullable = true)
 |-- add_on: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- add_on_description: string (nullable = true)
 |    |    |-- add_on_description_list: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- add_on_information_title: string (nullable = true)
 |    |    |-- add_on_information_url: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- is_default_select_addon: long (nullable = true)
 |    |    |-- is_visible: boolean (nullable = true)
 |    |    |-- list_price: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- price: long (nullable = true)
 |    |    |-- thumbnail_url: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- add_on_title: string (nullable = true)
 |-- all_time_quantity_sold: long (nullable = true)
 |-- asa_cashback_widget: struct (nullable = true)
 |   

48815

In [4]:
field_keep = ['day_ago_created', 'discount_rate', 'favourite_count', 'categories', 'name', 'id',
              'quantity_sold', 'rating_average', 'review_count','all_time_quantity_sold'];
df = df.select(*field_keep)
df = df.withColumn("category_id", col("categories.id")) \
        .withColumn("category_name", col("categories.name"))
df = df.withColumn("quantity_sold_value", col("quantity_sold.value"))
df = df.drop("categories", "quantity_sold")

In [5]:
df.printSchema()

root
 |-- day_ago_created: long (nullable = true)
 |-- discount_rate: long (nullable = true)
 |-- favourite_count: long (nullable = true)
 |-- name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- rating_average: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- all_time_quantity_sold: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_name: string (nullable = true)
 |-- quantity_sold_value: long (nullable = true)



In [6]:
# csv_output_path = "hdfs://localhost:9000/tiki/product_tiki.csv" 
# df.write.csv(csv_output_path, header=True, mode="overwrite")

In [7]:
# products = spark.read.csv("hdfs://localhost:9000/tiki/product_tiki.csv")
# products.printSchema()

In [8]:
ranked_products = df.withColumn(
    "ranking_score",
    (col("discount_rate") + col("rating_average") + col("favourite_count") +
     col("review_count") + col("all_time_quantity_sold") +
     col("quantity_sold_value"))
)

In [9]:
ranked_products = ranked_products.orderBy(col("ranking_score").desc())

In [ ]:
top_20_products = ranked_products.limit(20)
# Hiển thị bảng xếp hạng 20 sản phẩm đầu tiên
top_20_products.select("name", "id", "all_time_quantity_sold", "ranking_score").show()